In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
import pandas_profiling as ppf

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


test = pd.read_csv("dataset/test.csv")
train = pd.read_csv("dataset/train.csv")

### EDA

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
profile = ppf.ProfileReport(train)
profile.to_file(output_file="output.html")

### Data Preprocessing
#### Missing Values and Drop Unnecessary Cols
`Age` has 19.9% missing value, use mean to fill in the missing value.


In [4]:
train['Age'] = train.Age.fillna(np.mean(train.Age))
test['Age'] = test.Age.fillna(np.mean(test.Age))
train['Fare'] = train.Fare.fillna(np.mean(train.Fare))
test['Fare'] = test.Fare.fillna(np.mean(test.Fare))
train.drop(['Name','Cabin','Ticket','Embarked'],axis = 1, inplace = True)
test.drop(['Name','Cabin','Ticket','Embarked'],axis = 1, inplace = True)

In [5]:
test.Fare.isnull().sum() # double check how many missing values

0

#### Encode `gender` variable
`gender` is categorical variable and we want to convert it into numerical data.

In [6]:
lab = LabelEncoder()
train.Sex = lab.fit_transform(train.Sex)
test.Sex = lab.fit_transform(test.Sex)

#### Normalize Some of the Variables
Nornalize `Age`, `Fare`  by MinMaxScaler

In [7]:
minmax = MinMaxScaler()
train.Age = minmax.fit_transform(np.array(train.Age).reshape(-1,1))
test.Age = minmax.fit_transform(np.array(test.Age).reshape(-1,1))
train.Fare = minmax.fit_transform(np.array(train.Fare).reshape(-1,1))
test.Fare = minmax.fit_transform(np.array(test.Fare).reshape(-1,1))

In [8]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,1,0.452723,0,0,0.015282
1,893,3,0,0.617566,1,0,0.013663
2,894,2,1,0.815377,0,0,0.018909
3,895,3,1,0.353818,0,0,0.016908
4,896,3,0,0.287881,1,1,0.023984


In [9]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,0.271174,1,0,0.014151
1,2,1,1,0,0.472229,1,0,0.139136
2,3,1,3,0,0.321438,0,0,0.015469
3,4,1,1,0,0.434531,1,0,0.103644
4,5,0,3,1,0.434531,0,0,0.015713


In [10]:
x = train.drop('Survived',axis = 1)
y = train.Survived

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 0)

In [12]:
x_train.shape, y_train.shape

((623, 7), (623,))

### Model
Use logistic regression here.

In [21]:
from sklearn.linear_model import LogisticRegression

logr = LogisticRegression()
logr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [22]:
y_pred = logr.predict(x_test)

In [15]:
mae = mean_absolute_error(y_pred,y_test)
mae

0.2931077454852208

In [23]:
prediction = logr.predict(test)
prediction

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [24]:
submission = pd.DataFrame({'PassengerId':test.PassengerId, 'Survived':prediction})
submission.to_csv('submission_1.csv',index = False)

In [25]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
